In [ ]:
import numpy as np
import os
import pandas as pd
import requests

# Set the base directory to your target folder
base_directory = r'F:\\Persönliches\\Git\BioOTon\\XenoCanto'
os.makedirs(base_directory, exist_ok=True)

In [ ]:
# Get number of pages with the results
api_url = f"https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486"
print(f"→ Querying {api_url}")
response = requests.get(api_url)
if response.status_code != 200:
    print(f"API request failed.")

# process response to get page numbers
data = response.json()
page_number = data["numPages"]

recordings_list = []

# Build frame with all recording ids and lat / lon
for p in np.arange(1, page_number+1):
    api_url = f"https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page={p}"
    print(f"→ Querying {api_url}")
    response = requests.get(api_url)
    if response.status_code != 200:
        print(f"API request failed.")

    data = response.json()
    recordings = data.get('recordings', [])

    
    if not recordings:
        print(f"No recordings found.")

    recordings_list.append(recordings)

→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=1
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=2
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=3
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=4
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=5
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=6
→ Querying https://x

In [ ]:
new_df = pd.DataFrame(all_new_recordings)
a = pd.concat([a, new_df], ignore_index=True)

In [ ]:
# 1. Create a list to hold your new data
all_new_recordings = []

# 2. Add your dictionaries to the list (this is very fast)
all_new_recordings.append(recording_1)
all_new_recordings.append(recording_2)
# ... etc

# 3. Convert the list to a DataFrame and concat ONCE
new_df = pd.DataFrame(all_new_recordings)
a = pd.concat([a, new_df], ignore_index=True)

In [79]:
a = pd.DataFrame(recordings)

In [91]:
b = pd.DataFrame(recordings)

In [92]:
a = pd.concat([a, b], ignore_index=True)

In [76]:
recordings

[{'id': '1002946',
  'gen': 'Mystery',
  'sp': 'mystery',
  'ssp': '',
  'grp': 'birds',
  'en': 'Identity unknown',
  'rec': 'Philip',
  'cnt': 'Germany',
  'loc': 'Breisgau-Hochschwarzwald (near  Ihringen), Freiburg, Baden-Württemberg',
  'lat': '48.0511',
  'lon': '7.6451',
  'alt': '240',
  'type': 'song',
  'sex': 'uncertain',
  'stage': 'adult',
  'method': 'in the hand',
  'url': '//xeno-canto.org/1002946',
  'file': 'https://xeno-canto.org/1002946/download',
  'file-name': 'XC1002946-2025-05-30-1339_[cut_40sec].mp3',
  'sono': {'small': '//xeno-canto.org/sounds/uploaded/FVEHBYBLKF/ffts/XC1002946-small.png',
   'med': '//xeno-canto.org/sounds/uploaded/FVEHBYBLKF/ffts/XC1002946-med.png',
   'large': '//xeno-canto.org/sounds/uploaded/FVEHBYBLKF/ffts/XC1002946-large.png',
   'full': '//xeno-canto.org/sounds/uploaded/FVEHBYBLKF/ffts/XC1002946-full.png'},
  'osci': {'small': '//xeno-canto.org/sounds/uploaded/FVEHBYBLKF/wave/XC1002946-small.png',
   'med': '//xeno-canto.org/sounds/upl

In [69]:
data["recordings"]

[{'id': '1048296',
  'gen': 'Branta',
  'sp': 'bernicla',
  'ssp': 'bernicla',
  'grp': 'birds',
  'en': 'Brant Goose',
  'rec': 'Manuel Tacke',
  'cnt': 'Germany',
  'loc': 'Ostaußengroden, Wangerooge, Niedersachsen',
  'lat': '53.7787',
  'lon': '7.9598',
  'alt': '0',
  'type': 'call',
  'sex': 'uncertain',
  'stage': 'uncertain',
  'method': 'field recording',
  'url': '//xeno-canto.org/1048296',
  'file': 'https://xeno-canto.org/1048296/download',
  'file-name': 'XC1048296-20251013_1300_Brant_call_Wangerooge-Ostaußengroden.mp3',
  'sono': {'small': '//xeno-canto.org/sounds/uploaded/KCXYVLQJCE/ffts/XC1048296-small.png',
   'med': '//xeno-canto.org/sounds/uploaded/KCXYVLQJCE/ffts/XC1048296-med.png',
   'large': '//xeno-canto.org/sounds/uploaded/KCXYVLQJCE/ffts/XC1048296-large.png',
   'full': '//xeno-canto.org/sounds/uploaded/KCXYVLQJCE/ffts/XC1048296-full.png'},
  'osci': {'small': '//xeno-canto.org/sounds/uploaded/KCXYVLQJCE/wave/XC1048296-small.png',
   'med': '//xeno-canto.org/s

In [41]:
recordings

[{'id': '1051878',
  'gen': 'Capreolus',
  'sp': 'capreolus',
  'ssp': '',
  'grp': 'land mammals',
  'en': 'Western Roe Deer',
  'rec': 'Arjan Boonman',
  'cnt': 'Germany',
  'loc': 'Botanischer Garten und Botanisches Forschungslabor der Universität Konstanz (near  Konstanz), Freiburg, Baden-Württemberg',
  'lat': '47.693',
  'lon': '9.1785',
  'alt': '460',
  'type': 'call',
  'sex': 'uncertain',
  'stage': 'adult',
  'method': 'field recording',
  'url': '//xeno-canto.org/1051878',
  'file': 'https://xeno-canto.org/1051878/download',
  'file-name': 'XC1051878-20250409_193800roedeerclose.wav',
  'sono': {'small': '//xeno-canto.org/sounds/uploaded/BATZCVUEMZ/ffts/XC1051878-small.png',
   'med': '//xeno-canto.org/sounds/uploaded/BATZCVUEMZ/ffts/XC1051878-med.png',
   'large': '//xeno-canto.org/sounds/uploaded/BATZCVUEMZ/ffts/XC1051878-large.png',
   'full': '//xeno-canto.org/sounds/uploaded/BATZCVUEMZ/ffts/XC1051878-full.png'},
  'osci': {'small': '//xeno-canto.org/sounds/uploaded/BATZ

In [ ]:
total_downloaded = 0

for genus, species in species_list:
    # Folder names are in the format {genus}_{species}-TEST
    folder_name = f"{genus}_{species}-TEST"
    save_directory = os.path.join(base_directory, folder_name)
    os.makedirs(save_directory, exist_ok=True)
    print(f"\n🔍 Downloading recordings for {genus} {species} into {save_directory}")
    
    # Determine starting page:
    # For species in original_species, skip page 1 (i.e., start from page 2)
    # For new species, download starting at page 1
    # This exists because I had downloaded page 1 of a few species and wanted to download the other pages as well as add some other new species
    # Please change the start_page for original and new species as you please (you may want to start originals (already downloaded) from page 2 for example
    if (genus, species) in original_species:
        start_page = 1
        # New species here:
    else:
        start_page = 1
    
    page = start_page
    while True:
      
        api_url = f"https://xeno-canto.org/api/2/recordings?query={genus}+{species}&page={page}"
        print(f"→ Querying {api_url}")
        response = requests.get(api_url)
        if response.status_code != 200:
            print(f"API request failed for {genus} {species} on page {page}")
            break
        
        data = response.json()
        recordings = data.get('recordings', [])
        if not recordings:
            print(f"No recordings found for {genus} {species} on page {page}")
            break
        
        for recording in recordings:
            try:
                recording_id = recording['id']
                # Download URL using the recording ID
                file_url = f"https://xeno-canto.org/{recording_id}/download"
                filename = f"{genus}_{species}_{recording_id}.wav"
                save_path = os.path.join(save_directory, filename)
                
                with requests.get(file_url, stream=True) as r:
                    if r.status_code == 200:
                        with open(save_path, 'wb') as f:
                            for chunk in r.iter_content(chunk_size=8192):
                                f.write(chunk)
                        print(f"Downloaded {filename}")
                        total_downloaded += 1
                    else:
                        print(f"Failed to download {filename} (status: {r.status_code})")
            except Exception as e:
                print(f"Error downloading recording {recording.get('id', '?')}: {e}")
        
        # Check the maximum number of pages from the API response
        num_pages = int(data.get('numPages', 1))
        if page >= num_pages:
            print(f"Reached the last page ({page}) for {genus} {species}.")
            break
        else:
            page += 1
            print(f"Moving to page {page} for {genus} {species}.")
            
print(f"\nFinished downloading. Total files downloaded: {total_downloaded}")
